# Baseline : Food-drug Interaction Project

# Step 1: Load food names or compounds into a list of unique items

In [1]:
import pandas as pd
import numpy as np
import json
import pickle
import json
import jellyfish

# UTF-8 support
import codecs

# To use PubMed API
import pubmed.utils as pb

# Split abstracts to sentences
from nltk.tokenize import sent_tokenize

import random

### Functions for Step 1

In [196]:
def replaceNonASCII(text):
    ''' Replace all non ASCII characters by 'unk'
        This is to deal with Unicode encode/decode bug when using PySpark
    '''
    if text is not None:
        return ''.join([i if ord(i) < 128 else 'unk' for i in text])

    
def list_loader(data):
    ''' Load pickle file
        Returns keys of the loaded dictionary as a set
    '''
    test = pickle.load(open( data , "rb"))
    foodlist = []
    for food in test.keys():
        food = replaceNonASCII(food.decode('utf-8').lower())
        foodlist.append(food)
        
    return set(foodlist)  # Will work with a set rather than a list. Faster search for later (hash)

### Using Adam's pickle file (food common name as sample)

For pickling the data from foodb.ca database, see Adam's notebook **compound_food_id.ipynb**  

As a first test, we will use only the food common name (not scientific name) only. Compounds names will be added once this test passes.

In [3]:
# food_common.pickle: Dictionary with common English food names as keys, compounds as values
test = pickle.load(open( "data/food_common.pickle", "rb"))

In [4]:
# Quick peek at the loaded data
for i, item in enumerate(test.iteritems()):
    if i == 0:
        print 'food name'
        print '-'*10
    print '{0}'.format(item[0])
    if i == 10: break

food name
----------
Oregon yampah
Okra
Black mulberry
Avocado
Parsley
Elderberry
Sugar
Sweet bay
Common bean
Fig
Lard


In [104]:
foodlist = list_loader('data/food_common.pickle')

In [197]:
i = 0
for food in foodlist:
    print food
    i += 1
    if i > 5:
        break

atlantic pollock
mixed nuts
rose hip
black mulberry
pheasant
whiting


# Step 2: Filter sentences from abstract with drug keyword and food names

Note: Test with only the abstracts' first json file. Once works, we can add all 100 remaining files.

In [11]:
import pyspark
from pyspark import SparkContext
sc = SparkContext()

### Functions for Step 2

In [198]:
def abstract_loader(name):
    ''' Loads abstracts from json files in provided list
    '''
    data_list = []
    for jsonfile in name:
        with codecs.open(jsonfile,"r","utf-8") as data_file:
            data = json.load(data_file)
            values = [replaceNonASCII(value) for value in data.values()]
            data_list.extend(values)
            
    return data_list


def splitSentences(abstract):
    sentences = sent_tokenize(abstract)
    return sentences


def find_ngrams(sentence, n):
    ''' Return list of ngrams from a sentence
    '''
    words_list = sentence.split()
    ngrams = zip(*[words_list[i:] for i in range(n)])
    #return [''.join([unicode(w)+' ' for w in ngram if type(w)==unicode]).strip() for ngram in ngrams]
    return [''.join([w+' ' for w in ngram]).strip() for ngram in ngrams]


#######################
# Method 1 - NOT USED #
#######################

# NOT USED since will return True if finds a food name within a word in the sentence
# E.g.: "pie" food name and "therapies" word in sentence: return True since "pie" in "therapies"
#def includeFoodCmpd(sentence, fdlist):
#    if any(word in sentence for word in fdlist):
#        return True
#    else:
#        return False


###################################
# Method 2 - solution to method 2 #
###################################

def includeFoodCmpd(sentence, fdlist, limit, verbose=False):
    ''' Calculates the Jaro Wrinkler distance between food name and ngrams in the sentence.
        Returns True if distance > 0.95
    '''
    result = False
    for food in fdlist:
        n = min(3, len(food.split()))  # Assuming max as trigram        
        sentence_ngrams = find_ngrams(sentence, n)  # Note: punctuation at end of sentence will be included with
                                                # last word. For now ok, since the JW will still be > 0.95
        for ngram in sentence_ngrams:
            # Note: when using jaro_winkler, need to convert into unicode format
            ngram_distance = jellyfish.jaro_winkler(food.lower(), u"{}".format(ngram.lower()))
            if verbose:
                print food, ngram, ngram_distance
            if ngram_distance > limit:  
                result = True
                break

    return result


def findFood(sentence, foods):
    ''' Returns the food names found in the sentence based on string distance method
    '''
    
    food = []

    # If using string distance method:
    for f in foods:
        n = min(3, len(f.split()))  # Assuming max as trigram        
        sentence_ngrams = find_ngrams(sentence, n)  # Note: punctuation at end of sentence will be included with
                                                # last word. For now ok, since the JW will still be > 0.95
        for ngram in sentence_ngrams:
            # Note: when using jaro_winkler, need to convert into unicode format
            if jellyfish.jaro_winkler(f.lower(), u"{}".format(ngram.lower())) > 0.95:
                food.append(f)
            
    return food


# Not used since we will use the string distance for filtering
def findFoodItems(sentence, foods):
    for item in foods:
        if item in sentence:
            print item

In [199]:
# Test find_ngrams:
string = u"We are working hard on 266 project baseline"
find_ngrams(string, 3)

[u'We are working',
 u'are working hard',
 u'working hard on',
 u'hard on 266',
 u'on 266 project',
 u'266 project baseline']

In [200]:
# Testing includeFoodCmpd function
ss = u'Panda is eating a pie'
testlist = [u"hello",u"pie"]
includeFoodCmpd(ss, testlist, 0.98, verbose=True)


hello Panda 0.0
hello is 0.0
hello eating 0.455555555556
hello a 0.0
hello pie 0.511111111111
pie Panda 0.511111111111
pie is 0.0
pie eating 0.5
pie a 0.0
pie pie 1.0


True

In [166]:
# Load all abstracts - there are 101 json files from PubMed
filename_list = []
for i in xrange(1, 102):
    filename_temp = "pbabstract" + str(i)+ ".json"
    filename_list.append(filename_temp)

abstract = abstract_loader(filename_list)

In [183]:
abstractRDD = sc.parallelize(abstract)\
                .filter(lambda a: a is not None)  #Some lines were empty: need to filter out
abstractRDD.take(2)

[u'Heart failure still has a significant disease burden with poor outcomes worldwide despite advances in therapy. The standard therapies have been focused on blockade of renin-angiotensin-aldosterone system with angiotensin-converting enzyme inhibitors, angiotensin receptor blockers and mineralocorticoid antagonists and the sympathetic nervous system with unk-blockers. The natriuretic peptide system is a potential counter-regulatory system that promotes vasodilatation and natriuresis. Angiotensin receptor neprilysin inhibitors are a new class drug capable of blocking the renin-angiotensin-aldosterone system and enhancing the natriuretic peptide system to improve neurohormonal balance. The success of the PARADIGM-HF trial with LCZ696 and its approval for heart failure treatment is likely to generate a paradigm shift. This review summarises the current knowledge of LCZ696 with a focus on pharmacology, pharmacokinetics and pharmacodynamics, mechanisms of action, clinical efficacy and safe

In [330]:
# Note: tried to braodcast the foodlist but got an error message when used it in below filter
# "TypeError: 'Broadcast' object is not iterable".... any idea why?
# foodlist_bcast = sc.broadcast(foodlist)

In [184]:
drugkeyword = "ACEI"

sentences = abstractRDD.flatMap(splitSentences) \
                       .map(lambda a: pb.ace_substitutor(a, drugkeyword)) \
                       .filter(lambda a: drugkeyword in a)\
                       .filter(lambda a: includeFoodCmpd(a, foodlist, 0.98))

In [174]:
sentences.take(2)

[u'in this study, we examined the separated caseins and whey proteins of goat milk for the presence of ACEI inhibitory peptides.',
 u'digestion of isolated whey proteins and caseins of goat milk by gastric pepsin generated soluble hydrolysates exhibiting significant inhibition of ACEI compared to weak inhibition by undigested proteins.']

In [185]:
%%time
filtered_sentences = sentences.collect()

CPU times: user 71.2 ms, sys: 30.3 ms, total: 101 ms
Wall time: 21min 30s


In [186]:
len(filtered_sentences)

938

In [194]:
# Save filtered sentences containing drug and food name in textfile
with open("data/FilteredSentences.txt", "w") as outcomes:
    for sentence in filtered_sentences:
        food = findFood(sentence, foodlist)
        outcomes.writelines(str(food) + sentence + "\n")

## Step 3: Classify sentences as positive or negative based on a sentiment lexicon

There are 938 sentences containing both ACE Inhibitor drug and a food name from Step 2. They will be labeled and then compared to the 2 baseline models classification for model evaluation.

## BASELINE 1: 50/50 coin flip classification

Our first model is to simply classify the interaction between drug and food in the sentence as positive, negative or neutral based on equal probability of 1/3 in each class.  

Evaluation metrics: the labels of 100 randomly chosen sentences from outcomes in step 2 (i.e. sentences with drug and food names) will be compared with the random labels from baseline model.

In [206]:
classification = [(random.choice(["positive", "negative", "neutral"]), sentence) for sentence in filtered_sentences]

## BASELINE 2: Sentiment lexicon

Sentiment lexicon used is the Harvard General Inquirer (http://www.wjh.harvard.edu/~inquirer/spreadsheet_guide.htm). It contains 1,915 positive words and 2,291 negative words and is free for research use.

In [208]:
db_sentiment = pd.read_csv("data/inquirerbasic.csv", encoding = 'utf8')

/Applications/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [209]:
db_sentiment.head()

,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined
0,A,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET ART,| article: Indefinite singular article--some o...
1,ABANDON,H4Lvd,NaN,Negativ,NaN,NaN,Ngtv,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
2,ABANDONMENT,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
3,ABATE,H4Lvd,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
4,ABATEMENT,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,NaN


In [210]:
# Look at all data fields available
for column in db_sentiment.columns:
    print column

Entry
Source
Positiv
Negativ
Pstv
Affil
Ngtv
Hostile
Strong
Power
Weak
Submit
Active
Passive
Pleasur
Pain
Feel
Arousal
EMOT
Virtue
Vice
Ovrst
Undrst
Academ
Doctrin
Econ@
Exch
ECON
Exprsv
Legal
Milit
Polit@
POLIT
Relig
Role
COLL
Work
Ritual
SocRel
Race
Kin@
MALE
Female
Nonadlt
HU
ANI
PLACE
Social
Region
Route
Aquatic
Land
Sky
Object
Tool
Food
Vehicle
BldgPt
ComnObj
NatObj
BodyPt
ComForm
COM
Say
Need
Goal
Try
Means
Persist
Complet
Fail
NatrPro
Begin
Vary
Increas
Decreas
Finish
Stay
Rise
Exert
Fetch
Travel
Fall
Think
Know
Causal
Ought
Perceiv
Compare
Eval@
EVAL
Solve
Abs@
ABS
Quality
Quan
NUMB
ORD
CARD
FREQ
DIST
Time@
TIME
Space
POS
DIM
Rel
COLOR
Self
Our
You
Name
Yes
No
Negate
Intrj
IAV
DAV
SV
IPadj
IndAdj
PowGain
PowLoss
PowEnds
PowAren
PowCon
PowCoop
PowAuPt
PowPt
PowDoct
PowAuth
PowOth
PowTot
RcEthic
RcRelig
RcGain
RcLoss
RcEnds
RcTot
RspGain
RspLoss
RspOth
RspTot
AffGain
AffLoss
AffPt
AffOth
AffTot
WltPt
WltTran
WltOth
WltTot
WlbGain
WlbLoss
WlbPhys
WlbPsyc
WlbPt
WlbTot
EnlGain
EnlLo

**Note:** some columns seem quite interesting for analysing relationship other than simply positive or negative sentiment (e.g. "causal", etc.). For the baseline, we will only use the "positive" and "negative" columns

In [211]:
# Filter only the words labeled positive or negative
positive = db_sentiment[db_sentiment.Positiv == "Positiv"].Entry.map(lambda x: x.lower()).tolist()
negative = db_sentiment[db_sentiment.Negativ == "Negativ"].Entry.map(lambda x: x.lower()).tolist()

In [212]:
positive[:4]

[u'abide', u'ability', u'able', u'abound']

In [213]:
negative[:4]

[u'abandon', u'abandonment', u'abate', u'abdicate']

In [214]:
# Transform list into sets for faster search
positive = set(positive)
negative = set(negative)

Below is an attempt to classify whether a sentence is positive or negative.  
Note the main weaknesses:  
1. It is "positively" biased for now since looks at the positive words first and if it finds it, then it immediately returns positive. Thus, it may not look at the entire sentence in case of both positive or negative words.  
2. Negation of a positive word is not taken into account

In [215]:
def includeSentiment(sentence, poslist, neglist): 
    ''' Classify sentence as positive or negative based on first word found
        in the lexicon
    '''
    if any(word in sentence for word in poslist):
        return ("positive", sentence)
    elif any(word in sentence for word in neglist):
        return ("negative", sentence)
    else:
        return ("neutral", sentence)

In [216]:
# Add sentiment as key in the RDD

filtered_sentencesRDD = sc.parallelize(filtered_sentences)
sentiments = filtered_sentencesRDD.map(lambda a: includeSentiment(a, positive, negative))


In [217]:
# Peek at 2 first lines
sentiments.take(2)

[('positive',
  u'in this study, we examined the separated caseins and whey proteins of goat milk for the presence of ACEI inhibitory peptides.'),
 ('positive',
  u'digestion of isolated whey proteins and caseins of goat milk by gastric pepsin generated soluble hydrolysates exhibiting significant inhibition of ACEI compared to weak inhibition by undigested proteins.')]

In [218]:
# Create 2 lists of filtered sentences: one positive list and one negative list
pos_sentiments = sentiments.lookup("positive")
neg_sentiments = sentiments.lookup("negative")

In [219]:
# Examples of sentences with positive sentiment lexicon
pos_sentiments[:10]

[u'in this study, we examined the separated caseins and whey proteins of goat milk for the presence of ACEI inhibitory peptides.',
 u'digestion of isolated whey proteins and caseins of goat milk by gastric pepsin generated soluble hydrolysates exhibiting significant inhibition of ACEI compared to weak inhibition by undigested proteins.',
 u'the peptides from whey and caseins exert significant ACEI inhibitory activities comparable to that of captopril, an antihypertensive drug, exhibiting ic50 values of 4.45unkunkm and 4.27unkunkm, respectively.',
 u'the results introduce, for the first time, new potent ACEI-inhibitory peptides that can be released by gastric pepsin of goat milk whey and caseins and thus may pave the way for their candidacy as anti-hypertensive bioactive peptides and prevention of associated disorders.',
 u'ACEI use between 1 january 2003 and the index date were determined by the date of hospitalization for acute pancreatitis among the cases.',
 u'compared to hydrolysat

In [220]:
# Examples of sentences with negative sentiment lexicon
neg_sentiments[:2]

[u'the late-eluting fraction (f4) of either whey or caseins exhibited greater ACEI inhibition.',
 u'we investigated the molecular mechanisms involved in the ACEI (ACEI) inhibition by (-)-epigallocatechin-3-gallate (egcg), a major tea catechin.']

In [221]:
def findSentiment(sentence, sentiment, poslist, neglist):
    '''Print out the lexicon word that classified the sentence as positive or negative
    '''
    if sentiment == "positive":
        lexicon = poslist
    else:
        lexicon = neglist
    for word in lexicon:
        # Note: had to use this "try/except" since there was an unicode ascii error... any ways
        # to fix this without this try/except? if left the same, then we won't be able to see
        # some sentiment word in some sentences.
        try:
            if word in sentence:
                print word
        except:
            next

In [223]:
s5 = u"digestion of isolated whey proteins and caseins of goat milk by gastric pepsin generated soluble hydrolysates exhibiting significant inhibition of ACEI compared to weak inhibition by undigested proteins"
print findFood(s5, foodlist)
findSentiment(s5, "positive", positive, negative)

[u'casein', u'whey']
pro
generate
significant


In [224]:
s6 = u"the late-eluting fraction (f4) of either whey or caseins exhibited greater ACEI inhibition"
print findFood(s6, foodlist)
findSentiment(s6, "negative", positive, negative)

[u'casein', u'whey']
inhibit
inhibition


In [225]:
def findTags(sentence, sentiment, foods, poslist, neglist, limit):
    ''' Returns the tags of the sentence 
        Both lexicon word that classified the sentence as positive or negative and food name
    '''
    
    sent = []
    food = []
    
    if sentiment == "positive":
        lexicon = poslist
    else:
        lexicon = neglist
    for word in lexicon:
        # Note: had to use this "try/except" since there was an unicode ascii error... any ways
        # to fix this without this try/except? if left the same, then we won't be able to see
        # some sentiment word in some sentences.
        try:
            if word in sentence:
                sent.append(word)
        except:
            next
            
    # If using simple test of if food name "in" sentence method
    #for f in foods:
    #    if f in sentence:
    #        food.append(f)
    
    # If using string distance method:
    for f in foods:
        n = min(3, len(f.split()))  # Assuming max as trigram        
        sentence_ngrams = find_ngrams(sentence, n)  # Note: punctuation at end of sentence will be included with
                                                # last word. For now ok, since the JW will still be > 0.95
        for ngram in sentence_ngrams:
            # Note: when using jaro_winkler, need to convert into unicode format
            if jellyfish.jaro_winkler(f.lower(), u"{}".format(ngram.lower())) > limit:
                food.append(f)

            
    return [sent, food, sentence]

In [197]:
# Save results in a text file
# Note: this could have been also done in Spark! But felt lazy to code... feel free to try!

In [226]:
with open("data/Positive.txt", "w") as pos:
    for sentence in pos_sentiments:
        tags = findTags(sentence, "positive", foodlist, positive, negative, 0.95)
        pos.writelines(str(tags)+ "\n")
    
    

In [227]:
with open("data/Negative.txt", "w") as pos:
    for sentence in pos_sentiments:
        tags = findTags(sentence, "negative", foodlist, positive, negative, 0.95)
        pos.writelines(str(tags)+ "\n")

**Final Notes**  
1. In our baseline, some words like "date" that appear in a sentence will be interpreted as the fruit "date" instead of a calendar date and thus, will be filtered as outputs sentences. This can only be solved if we take into account the context of the sentence and we will need ML to model this!  
2. Sentiment analysis need a major improvement: only basing on the positive and negative words without how the food and drug are connected through these words is not a good model.